# Delta Robot Kinematics, Configuration Space, Workspace and Singularity Analysis

## Degrees of Freedom

The delta robot has 3 degrees of freedom. The 3 degrees of freedom are the 3 angles of the 3 arms. The 3 angles are the angles between the arms and the base. The 3 angles are denoted by $\theta_1$, $\theta_2$, and $\theta_3$. The 3 angles are the joint angles of the delta robot. The 3 angles are the configuration space of the delta robot. The configuration space of the delta robot is the space of all possible joint angles of the delta robot. The configuration space of the delta robot is a 3-dimensional space. The configuration space of the delta robot is a 3-dimensional space because the delta robot has 3 degrees of freedom.

<div style="text-align: center;">
  <img src="images/delta_robot_modern_robotics.png" alt="Delta Robot">
</div>

In [60]:
# Import block
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration Space

The delta robot's configuration space is bound by the 3 motor angles $\theta_1$, $\theta_2$, and $\theta_3$. The configuration space is a 3D space, where each point in the space corresponds to a unique position of the robot's end effector.

In [61]:
# The C-Space for the delta robot is 3 dimensional for the 3 motors

# Motor ranges in Dynamixel steps
theta1_range = [2100, 2850]
theta2_range = [2100, 2850]
theta3_range = [2100, 2850]

# Use a smaller number of points for a scatter plot to keep rendering fast
num_points = 20
theta1 = np.linspace(theta1_range[0], theta1_range[1], num_points)
theta2 = np.linspace(theta2_range[0], theta2_range[1], num_points)
theta3 = np.linspace(theta3_range[0], theta3_range[1], num_points)
Theta1, Theta2, Theta3 = np.meshgrid(theta1, theta2, theta3)

# Create a 3D scatter plot of the configuration space points using Plotly
fig = go.Figure(data=[go.Scatter3d(
  x=Theta1.flatten(),
  y=Theta2.flatten(),
  z=Theta3.flatten(),
  mode='markers',
  marker=dict(size=3, opacity=0.5)
)])

# Set the labels
fig.update_layout(scene=dict(
  xaxis_title='Theta1',
  yaxis_title='Theta2',
  zaxis_title='Theta3'
))

# Show the plot
fig.show()

## Robot Constants

In [62]:
# Robot Constants
SB = 261.306  # Base Equilateral Triangle Side Length [mm]
SP = 73.481  # Platform Equilateral Triangle Side Length [mm]
L = 108  # Active Link Length [mm]
ELL = 160  # Passive Link Length [mm]
H = 32  # Passive Link Width [mm]
WB = (np.sqrt(3) / 6) * SB
UB = (np.sqrt(3) / 3) * SB
WP = (np.sqrt(3) / 6) * SP
UP = (np.sqrt(3) / 3) * SP

# Inverse Kinematics
https://people.ohio.edu/williams/html/PDF/DeltaKin.pdf

In [63]:
def IK(x,y,z):
  # The analytical solution for IK uses the constraint equations found from the vector-loop closure equations
  # E_i * cos(theta_i) + F_i *  sin(theta_i) + G_i = 0
  # Variables obtained from the Vector-Loop Closure Equations
  a = WB - UP
  b = (SP / 2) - ((np.sqrt(3) / 2) * WP)
  c = WP - (WB / 2)
  E = (
    2*L*(y+a), # E1
    -L*(np.sqrt(3)*(x+b)+y+c),  # E2
    L*(np.sqrt(3)*(x-b)-y-c)  # E3
    )
  F = (
    2*z*L, # F1
    2*z*L,  # F2
    2*z*L,  # F3
  )
  G = (
      x**2 + y**2 + z**2 + a**2 + L**2 + 2*y*a - ELL**2,  # G1
      x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(x*b+y*c) - ELL**2,  # G2
      x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(-x*b+y*c) - ELL**2,  # G3
  )
  thetas = []
  # Tangent Half-Angle Substitution
  for i in range(3):
    D = E[i]**2 + F[i]**2 - G[i]**2
    t_plus = (-F[i] + np.sqrt(D)) / (G[i] - E[i])
    t_minus = (-F[i] - np.sqrt(D)) / (G[i] - E[i])
    theta_plus = 2 * np.arctan(t_plus)
    theta_minus = 2 * np.arctan(t_minus)
    # Pick the solution with the knees "kinked" out, the angle should be closer to 0
    thetas.append(theta_plus if abs(theta_plus) < abs(theta_minus) else theta_minus)
  # return a tuple of the 3 joint angles
  return np.array([thetas[0], thetas[1], thetas[2]])

up_position = IK(0, 0, -85)
print(f'IK(0,0,-85): {up_position}')
down_position = IK(0, 0, -200)
print(f'IK(0,0,-200): {down_position}')
# Test the IK function with a list of points
# Straight-up and down between Z=-200 and Z= -85
straight_up_down = [(0, 0, z) for z in np.linspace(-85, -200, 10)]
theta_values = np.array([IK(*point) for point in straight_up_down])
print(f'Trajectory from Z=-200 to Z=-85: {theta_values}')
assert theta_values.shape == (10, 3), 'Expected 10x3 array of joint angles'
assert np.allclose(theta_values[0], up_position, atol=1e-6), f'Expected first point to be up_position: {theta_values[0]} != {up_position}'
assert np.allclose(theta_values[-1], down_position, atol=1e-6), f'Expected last point to be down_position: {theta_values[-1]} != {down_position}'

IK(0,0,-85): [ 0.08096934 -0.42716773 -0.42716773]
IK(0,0,-200): [0.83251966 0.62665617 0.62665617]
Trajectory from Z=-200 to Z=-85: [[ 0.08096934 -0.42716773 -0.42716773]
 [ 0.17762057 -0.25685451 -0.25685451]
 [ 0.26510382 -0.1152702  -0.1152702 ]
 [ 0.34735688  0.00867527  0.00867527]
 [ 0.4267854   0.12132895  0.12132895]
 [ 0.50506351  0.2267571   0.2267571 ]
 [ 0.58351917  0.32783793  0.32783793]
 [ 0.66336108  0.4268342   0.4268342 ]
 [ 0.74585513  0.52575484  0.52575484]
 [ 0.83251966  0.62665617  0.62665617]]


In [64]:
# Trajectory: An XY Circle with a center of (0, 0, -200) mm and a radius of 50 mm
# At the same time, the Z displacement goes through 2 cycles of a sine wave
# centered on Z = -200 mm with an amplitude of 25 mm
num_samples = 100
t = np.linspace(0, 2 * np.pi, num_samples)
x_circle = 50 * np.cos(t)
y_circle = 50 * np.sin(t)
# Two sine cycles in z with amplitude 25 mm
z_circle = -180 + 25 * np.sin(2 * t)
thetas_circle = []
for x, y, z in zip(x_circle, y_circle, z_circle):
  try:
    thetas_circle.append(IK(x, y, z))
  except Exception as e:
    print(f"Exception for point ({x}, {y}, {z}): {e}")
thetas_circle = np.array(thetas_circle)

# Plot the trajectory of the motor angles
fig = make_subplots(rows=1, cols=1, subplot_titles=('Motor Angles for XY Circle Trajectory'))

# Plot the theta values
fig.add_trace(go.Scatter(y=thetas_circle[:, 0], mode='lines', name='Theta1'), row=1, col=1)
fig.add_trace(go.Scatter(y=thetas_circle[:, 1], mode='lines', name='Theta2'), row=1, col=1)
fig.add_trace(go.Scatter(y=thetas_circle[:, 2], mode='lines', name='Theta3'), row=1, col=1)

fig.update_layout(height=600, width=800, title_text="Motor Angles for XY Circle Trajectory", xaxis_title='Sample', yaxis_title='Motor Angle (radians)')
fig.show()

# Forward Kinematics


In [65]:
def FK(th1,th2,th3):
  def virtual_sphere_centers(th1, th2, th3):
    # thetas are the joint angles [rad]
    return (
      (0, -WB-L*np.cos(th1)+UP, -L*np.sin(th1)), # Sphere 1
      ((np.sqrt(3)/2)*(WB+L*np.cos(th2))-(SP/2), (1/2) * (WB+L*np.cos(th2))-WP, -L*np.sin(th2)),  # Sphere 2
      (-(np.sqrt(3)/2)*(WB+L*np.cos(th3))+(SP/2), (1/2) * (WB+L*np.cos(th3))-WP, -L*np.sin(th3)),  # Sphere 3
    )
  # First determine the centers of the 3 virtual spheres
  sphere_centers = virtual_sphere_centers(th1, th2, th3)
  x1, y1, z1 = sphere_centers[0]
  x2, y2, z2 = sphere_centers[1]
  x3, y3, z3 = sphere_centers[2]
  
  # Determine if the all Z heights of the 3 spheres are the same to determine which algorithm to use
  same_z_height = all([center[2] == sphere_centers[0][2] for center in sphere_centers])
  
  if same_z_height:
    # Simplified 3 Sphere Intersection Algorithm
    assert z1 == z2 == z3
    z_n = z1 # All Z heights are the same
    a = 2*(x3 - x1)
    b = 2*(y3 - y1)
    c = ELL**2 - ELL**2 - x1**2 - y1**2 + x3**2 + y3**2
    d = 2*(x3 - x2)
    e = 2*(y3 - y2)
    f = ELL**2 - ELL**2 - x2**2 - y2**2 + x3**2 + y3**2
    denominator = (a*e - b*d)
    if denominator == 0:
      raise ValueError('FK(S): Denominator is zero, the spheres do not intersect')
    x = (c*e - b*f) / denominator
    y = (a*f - c*d) / denominator
    B = -2 * z_n
    C = z_n**2 - ELL**2 + (x - x1)**2 + (y - y1)**2
    discriminant = B**2 - 4*C
    if discriminant < 0:
      raise ValueError('FK(S): No real solutions, the spheres do not intersect')
    z_plus = (-B + np.sqrt(discriminant)) / 2
    z_minus = (-B - np.sqrt(discriminant)) / 2
    # Choose the Z value that is in the correct workspace (-Z)
    z = z_plus if z_plus < 0 else z_minus
    return np.array([x,y,z])
  else:
    # 3 Sphere Intersection Algorithm
    a11 = 2*(x3-x1)
    a12 = 2*(y3-y1)
    a13 = 2*(z3-z1)
    a21 = 2*(x3-x2)
    a22 = 2*(y3-y2)
    a23 = 2*(z3-z2)
    b1 = ELL**2 - ELL**2 - x1**2 - y1**2 - z1**2 + x3**2 + y3**2 + z3**2
    b2 = ELL**2 - ELL**2 - x2**2 - y2**2 - z2**2 + x3**2 + y3**2 + z3**2
    # Verify we aren't dividing by zero
    if a13 == 0 or a23 == 0:
      raise ValueError('FK: Division by zero, the spheres do not intersect')
    a1 = (a11/a13) - (a21/a23)
    a2 = (a12/a13) - (a22/a23)
    a3 = (b2/a23) - (b1/a13)
    a4 = -(a2/a1)
    a5 = -(a3/a1)
    a6 = (-a21*a4-a22)/a23
    a7 = (b2-a21*a5)/a23
    a_quad = a4**2 + 1 + a6**2
    b_quad = 2*a4*(a5-x1) - 2*y1 + 2*a6*(a7-z1)
    c_quad = a5*(a5-2*x1) + a7*(a7-2*z1) + x1**2 + y1**2 + z1**2 - ELL**2
    d_quad = b_quad**2 - 4*a_quad*c_quad
    if d_quad < 0:
      raise ValueError('FK: No real solutions, the spheres do not intersect')
    y_plus = (-b_quad+np.sqrt(d_quad))/(2*a_quad)
    x_plus = a4*y_plus + a5
    z_plus = a6*y_plus + a7
    y_minus = (-b_quad-np.sqrt(d_quad))/(2*a_quad)
    x_minus = a4*y_minus + a5
    z_minus = a6*y_minus + a7
    # Choose the Z value that is in the correct workspace (-Z)
    if z_plus < 0:
      z = z_plus
      x = x_plus
      y = y_plus
    else:
      z = z_minus
      x = x_minus
      y = y_minus
    return np.array([x, y, z])


print(f'FK with equal joint angles (-0.5) [rad]: {np.round(FK(-0.5, -0.5, -0.5), 4)} [mm]')
print(f'FK with different joint angles (-0.25, -0.5, -0.5) [rad]: {np.round(FK(-0.25, -0.5, 0.5),4)} [mm]')

FK with equal joint angles (-0.5) [rad]: [  0.       0.     -44.5051] [mm]
FK with different joint angles (-0.25, -0.5, -0.5) [rad]: [ 30.4833 -10.2531 -65.153 ] [mm]


In [72]:
# Testing

fd = np.deg2rad(15)
print(f'FK(15,15,15): {np.round(FK(fd,fd,fd), 4)} [mm]')

print(f'IK(0,0,-100): {np.round(IK(0,0,-100), 4)} [rad]')
print(f'IK(0,0,-200): {np.round(IK(0,0,-200), 4)} [rad]')

down = np.deg2rad(90)
print(f'FK(0,0,0): {np.round(FK(0,0,0), 4)} [mm]')
print(f'FK(90,90,90): {np.round(FK(down,down,down), 4)} [mm]')

FK(15,15,15): [  -0.        0.     -110.0557] [mm]
IK(0,0,-100): [ 0.1934 -0.2306 -0.2306] [rad]
IK(0,0,-200): [0.8325 0.6267 0.6267] [rad]
FK(0,0,0): [  0.      -0.     -75.6086] [mm]
FK(90,90,90): [  -0.       -0.     -264.5581] [mm]


# Plotting the Robot's Configuration

In order to better visualize the robot's configuration, we can plot the robot's links and joints in 3D space.

In [71]:
def motor_positions():
  return (
    (0,-WB,0), # B1
    ((np.sqrt(3)/2)*WB,WB/2,0), # B2
    (-(np.sqrt(3)/2)*WB, WB/2, 0)  # B3
  )
  
def base_vertices():
  return (
    (SB/2, -WB, 0), # b1
    (0, UB, 0), # b2
    (-SB/2, -WB, 0) # b3
  )

def platform_vertices(platform_position):
  # platform_position is a tuple (x,y,z)
  x, y, z = platform_position
  # Platform vertices with respect to the given platform position
  return (
    (x, y - UP, z), # P1
    (x + SP/2, y + WP, z), # P2
    (x - SP/2, y + WP, z) # P3
  )

def knee_joints(theta1,theta2,theta3):
  knee_1 = (0, -WB-L*np.cos(theta1), -L*np.sin(theta1))
  knee_2 = ((np.sqrt(3)/2)*(WB+L*np.cos(theta2)), (1/2)*(WB+L*np.cos(theta2)), -L*np.sin(theta2))
  knee_3 = ((-np.sqrt(3)/2)*(WB+L*np.cos(theta3)), (1/2)
            * (WB+L*np.cos(theta3)), -L*np.sin(theta3))
  return (knee_1, knee_2, knee_3)


def get_motor_rectangle_corners(motor_position, angle, motor_width=20, motor_length=40):
    # Local coordinates (centered at 0,0)
    dx = motor_length / 2
    dy = motor_width / 2
    local_corners = np.array([
        [-dx, -dy],
        [dx, -dy],
        [dx,  dy],
        [-dx,  dy]
    ])
    # Create rotation matrix for the given angle
    R = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])
    rotated = (local_corners @ R.T) + np.array(motor_position[:2])
    # Append the constant z coordinate from motor_position
    corners = [(pt[0], pt[1], motor_position[2]) for pt in rotated]
    return corners

def plot_robot(theta1, theta2, theta3):
  # Calculate the knee joint positions
  knee_1, knee_2, knee_3 = knee_joints(theta1, theta2, theta3)
  # Calculate the platform position
  try:
    platform_position = FK(theta1, theta2, theta3)
  except Exception as e:
    print(f"Exception for FK({theta1}, {theta2}, {theta3}): {e}")
    return
  # Get the base vertices
  bv1, bv2, bv3 = base_vertices()
  # Get the motor positions
  m1, m2, m3 = motor_positions()
  # Get the platform vertices
  pv1, pv2, pv3 = platform_vertices(platform_position)
  
  # Create a 3D plot
  fig = go.Figure()
  
  # --- Base Triangle (Mesh3d) ---
  base_x = [bv1[0], bv2[0], bv3[0]]
  base_y = [bv1[1], bv2[1], bv3[1]]
  base_z = [bv1[2], bv2[2], bv3[2]]
  fig.add_trace(go.Mesh3d(
      x=base_x, y=base_y, z=base_z,
      i=[0], j=[1], k=[2],
      color='lightblue', opacity=0.6,
      name='Base'
  ))
  
  # --- Platform Triangle (Mesh3d) ---
  plat_x = [pv1[0], pv2[0], pv3[0]]
  plat_y = [pv1[1], pv2[1], pv3[1]]
  plat_z = [pv1[2], pv2[2], pv3[2]]
  fig.add_trace(go.Mesh3d(
      x=plat_x, y=plat_y, z=plat_z,
      i=[0], j=[1], k=[2],
      color='lightgreen', opacity=1.0,
      name='Platform'
  ))
  
  # Draw a red sphere at the platform position
  fig.add_trace(go.Scatter3d(
        x=[platform_position[0]], y=[platform_position[1]], z=[platform_position[2]],
        mode='markers',
        marker=dict(color='red', size=10),
        name='Platform Center'
    ))

  # Add platform vertices as markers with labels
  platform_vertices_list = [pv1, pv2, pv3]
  for idx, pv in enumerate(platform_vertices_list, start=1):
      fig.add_trace(go.Scatter3d(
          x=[pv[0]], y=[pv[1]], z=[pv[2]],
          mode='markers+text',
          marker=dict(color='green', size=H/4),
          text=f'P{idx}',
          textposition='top center',
          name=f'PV {idx}',
          showlegend=False
      ))
  
  # --- Motor Rectangles ---
  # Determine motor orientations based on the base triangle sides:
  motor_angle1 = np.arctan2(bv1[1] - bv3[1], bv1[0] - bv3[0])
  motor_angle2 = np.arctan2(bv2[1] - bv1[1], bv2[0] - bv1[0])
  motor_angle3 = np.arctan2(bv3[1] - bv2[1], bv3[0] - bv2[0])
  motor_angles = [motor_angle1, motor_angle2, motor_angle3]
  motor_positions_list = [m1, m2, m3]
  motor_colors = ['red', 'green', 'blue']
  motor_ids = ['1', '2', '3']
  
# For each motor, compute its rectangle and add it as two triangles.
  for m, ang, color, mid in zip(motor_positions_list, motor_angles, motor_colors, motor_ids):
    corners = get_motor_rectangle_corners(m, ang)
    # Split quadrilateral into two triangles:
    # Triangle 1: corners[0], corners[1], corners[2]
    tri1_x = [corners[0][0], corners[1][0], corners[2][0]]
    tri1_y = [corners[0][1], corners[1][1], corners[2][1]]
    tri1_z = [corners[0][2], corners[1][2], corners[2][2]]
    
    # Triangle 2: corners[0], corners[2], corners[3]
    tri2_x = [corners[0][0], corners[2][0], corners[3][0]]
    tri2_y = [corners[0][1], corners[2][1], corners[3][1]]
    tri2_z = [corners[0][2], corners[2][2], corners[3][2]]
    
    # Add first triangle
    fig.add_trace(go.Mesh3d(
        x=tri1_x, y=tri1_y, z=tri1_z,
        color=color, opacity=0.7,
        name=f'Motor {mid}',
        showscale=False
    ))
    # Add second triangle
    fig.add_trace(go.Mesh3d(
        x=tri2_x, y=tri2_y, z=tri2_z,
        color=color, opacity=0.7,
        name=f'Motor {mid}',
        showscale=False
    ))
    # Add motor label as text at the motor position
    fig.add_trace(go.Scatter3d(
        x=[m[0]], y=[m[1]], z=[m[2]],
        mode='text',
        text=[mid],
        textposition='middle center',
        textfont=dict(color='black', size=18),
        showlegend=False
    ))
  # --- End of Motor Rectangles ---
    
  # --- Active Links (lines from motors to knee joints) ---
  knees = [knee_1, knee_2, knee_3]
  for m, k in zip(motor_positions_list, knees):
      fig.add_trace(go.Scatter3d(
          x=[m[0], k[0]],
          y=[m[1], k[1]],
          z=[m[2], k[2]],
          mode='lines',
          line=dict(color='purple', width=H/4),
          name='Active Link' + str(motor_positions_list.index(m) + 1),
          showlegend=False
      ))

      # --- Knee Joints as markers ---
      fig.add_trace(go.Scatter3d(
          x=[k[0]], y=[k[1]], z=[k[2]],
          mode='markers',
          marker=dict(color='black', size=10),
          name='Knee Joint' + str(knees.index(k) + 1),
          showlegend=False
      ))

  # --- Passive Links (rectangles from knee joints to platform vertices) ---
  for k, p in zip(knees, [pv1, pv2, pv3]):
      fig.add_trace(go.Scatter3d(
          x=[k[0], p[0]],
          y=[k[1], p[1]],
          z=[k[2], p[2]],
          mode='lines',
          line=dict(color='orange', width=H),
          name='Passive Link' + str(knees.index(k) + 1),
          showlegend=False
      ))
  
  
  # --- Update Layout and Style Plot ---
  fig.update_layout(
      scene=dict(
          xaxis_title='X [mm]',
          yaxis_title='Y [mm]',
          zaxis_title='Z [mm]',
          xaxis=dict(range=[-200, 200]),
          yaxis=dict(range=[-200, 200]),
          zaxis=dict(range=[-400, 0])
      ),
      title=f'Delta Robot Configuration (Theta1={theta1:.2f}, Theta2={theta2:.2f}, Theta3={theta3:.2f}) [rad]',
      width=1000,
      height=800
  )
  
  fig.show()


# "UP" position
plot_robot(
    theta1=np.deg2rad(15),
    theta2=np.deg2rad(15),
    theta3=np.deg2rad(15)
)

# "DOWN" position
plot_robot(
  theta1=np.pi/2,
  theta2=np.pi/2,
  theta3=np.pi/2
)

# Another configuration with different angles for each leg
plot_robot(
    theta1=np.pi/4 + 0.1,
    theta2=np.pi/4,
    theta3=np.pi/4 - 0.1
)

# Animating Robot Trajectories

In [68]:
def generate_robot_traces(theta1,theta2,theta3):
  # Calculate the knee joint positions
  knee_1, knee_2, knee_3 = knee_joints(theta1, theta2, theta3)
  # Calculate the platform position
  try:
    platform_position = FK(theta1, theta2, theta3)
  except Exception as e:
    print(f"Exception for FK({theta1}, {theta2}, {theta3}): {e}")
    return
  # Get the base vertices
  bv1, bv2, bv3 = base_vertices()
  # Get the motor positions
  m1, m2, m3 = motor_positions()
  # Get the platform vertices
  pv1, pv2, pv3 = platform_vertices(platform_position)

  base_triangle_trace = go.Mesh3d(
      x=[bv1[0], bv2[0], bv3[0]],
      y=[bv1[1], bv2[1], bv3[1]],
      z=[bv1[2], bv2[2], bv3[2]],
      i=[0], j=[1], k=[2],
      color='lightblue', opacity=0.6,
      name='Base'
  )
  
  platform_triangle_trace = go.Mesh3d(
      x=[pv1[0], pv2[0], pv3[0]],
      y=[pv1[1], pv2[1], pv3[1]],
      z=[pv1[2], pv2[2], pv3[2]],
      i=[0], j=[1], k=[2],
      color='lightgreen', opacity=1.0,
      name='Platform'
  )
  
  platform_center_trace = go.Scatter3d(
      x=[platform_position[0]], y=[platform_position[1]], z=[platform_position[2]],
      mode='markers', marker=dict(color='red', size=5), 
      name='Platform Center'
      )
  
  knee_traces = []
  for k in [knee_1, knee_2, knee_3]:
        knee_traces.append(go.Scatter3d(
            x=[k[0]], y=[k[1]], z=[k[2]],
            mode='markers',
            marker=dict(color='black', size=5),
            name='Knee Joint' + f'{[knee_1, knee_2, knee_3].index(k) + 1}',
            uid=f'KJ{[knee_1, knee_2, knee_3].index(k) + 1}',
            showlegend=False
        ))
        
  active_link_traces = []
  knees = [knee_1, knee_2, knee_3]
  for m, k in zip([m1, m2, m3], knees):
      active_link_traces.append(go.Scatter3d(
          x=[m[0], k[0]],
          y=[m[1], k[1]],
          z=[m[2], k[2]],
          mode='lines',
          line=dict(color='purple', width=H/6),
          name='Active Link' + f'{[m1, m2, m3].index(m) + 1}',
          uid=f'AL{[m1, m2, m3].index(m) + 1}',
          showlegend=False
      ))
  
  passive_link_traces = []
  for k, p in zip(knees, [pv1, pv2, pv3]):
      passive_link_traces.append(go.Scatter3d(
          x=[k[0], p[0]],
          y=[k[1], p[1]],
          z=[k[2], p[2]],
          mode='lines',
          line=dict(color='orange', width=H/2),
          name='Passive Link' + f'{knees.index(k) + 1}',
          uid=f'PL{knees.index(k) + 1}',
          showlegend=False
      ))
  
  return [base_triangle_trace, platform_triangle_trace, platform_center_trace] + knee_traces + active_link_traces + passive_link_traces

In [69]:
def animate_robot(trajectory: tuple[list[float], list[float], list[float]]):
    # trajectory is a tuple of (x,y,z) points
    x_t, y_t, z_t = trajectory
    assert len(x_t) == len(y_t) == len(z_t), "Trajectory points must have the same length"
    N = len(x_t)
    
    # Solve IK for the robot at each point of the trajectory
    theta_trajectory = np.array([IK(x, y, z) for x, y, z in zip(x_t, y_t, z_t)])
    
    trajectory_trace = go.Scatter3d(
        x=x_t,
        y=y_t,
        z=z_t,
        mode='lines',
        line=dict(color='blue', width=5),
        name='Trajectory'
    )
    
    # Build animation frames
    frames = []
    for i in range(num_samples):
        # Get joint anles for the current trajectory point using IK
        theta1, theta2, theta3 = theta_trajectory[i]
        # Generate traces for the robot at the current configuration
        robot_traces = generate_robot_traces(theta1, theta2, theta3)
        animation_traces = robot_traces + [trajectory_trace]
        # Create a frame with these traces
        frames.append(go.Frame(data=animation_traces, name=str(i)))

    # Create the initial figure using first trajectory point
    initial_theta1, initial_theta2, initial_theta3 = theta_trajectory[0]
    initial_robot_traces = generate_robot_traces(initial_theta1, initial_theta2, initial_theta3)
    initial_robot_traces.append(trajectory_trace)
    # Create the animation figure
    animation_speed = 5  # milliseconds per frame
    fig = go.Figure(data=initial_robot_traces, frames=frames)

    # Add animation buttons (Play/Pause) to the layout
    fig.update_layout(
        title="Delta Robot Animation",
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            {"frame": {"duration": animation_speed, "redraw": True},
                            "fromcurrent": True}
                        ]
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[
                            [None],
                            {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}
                        ]
                    )
                ],
                direction="left",
                pad={"r": 10, "t": 87},
                showactive=False,
                x=0.1,
                xanchor="right",
                y=0,
                yanchor="top"
            )
        ],
        sliders = [{
        "currentvalue": {"prefix": "Frame: "},
        "steps": [{
            "args": [
                [str(i)],
                {"frame": {"duration": N, "redraw": True}, "mode": "immediate"}
            ],
            "label": str(i),
            "method": "animate"
        } for i in range(N)]
        }],
        width=800,
        height=800,
        scene=dict(
            xaxis_title='X [mm]',
            yaxis_title='Y [mm]',
            zaxis_title='Z [mm]',
            xaxis=dict(range=[-200, 200], autorange=False),
            yaxis=dict(range=[-200, 200], autorange=False),
            zaxis=dict(range=[-400, 0], autorange=False)
        ),
        uirevision=True
    )

    fig.show()

In [70]:
# Create a simple up down trajectory between
# Z = -100 and Z = -200
num_samples = 100
z_traj = np.linspace(-100, -160, num_samples)
x_traj = np.zeros(num_samples)
y_traj = np.zeros(num_samples)

ee_trajectory = (x_traj, y_traj, z_traj)

for t in range(ee_trajectory[0].shape[0]):
  ik_result = IK(x_traj[t], y_traj[t], z_traj[t])
  fk_result = FK(*ik_result)
  assert np.allclose(fk_result, [x_traj[t], y_traj[t], z_traj[t]], atol=1e-6), f'FK(IK({x_traj[t]}, {y_traj[t]}, {z_traj[t]})) = {fk_result} != ({x_traj[t]}, {y_traj[t]}, {z_traj[t]})'

animate_robot(ee_trajectory)

ValueError: FK: Division by zero, the spheres do not intersect

In [ ]:

# Create the circle trajectory in the XY plane 
# while the Z coordinate goes through 2 cycles of a sine wave
circle_center_z = -180
amplitude_z = 25
num_samples = 500
t = np.linspace(0, 2 * np.pi, num_samples)
x_circle = 50 * np.cos(t)
y_circle = 50 * np.sin(t)
z_circle = circle_center_z + amplitude_z * np.sin(2 * t)

# First draw the 3D trajectory to visualize the path
fig = go.Figure()
fig.add_trace(go.Scatter3d(
  x=x_circle, y=y_circle, z=z_circle,
  mode='lines',
  line=dict(color='blue', width=2),
  name='Circle Trajectory'
))
fig.update_layout(
    scene=dict(
        xaxis_title='X [mm]',
        yaxis_title='Y [mm]',
        zaxis_title='Z [mm]',
        xaxis=dict(range=[-100, 100]),
        yaxis=dict(range=[-100, 100]),
        zaxis=dict(range=[-250, -150])
    ),
    title='XY Circle with Z Sine Wave Trajectory',
    width=600,
    height=600
)
fig.show()

# Pick a trajectory point
x_pt, y_pt, z_pt = x_circle[0], y_circle[0], z_circle[0]

# Compute the joint angles and platform position
theta1, theta2, theta3 = IK(x_pt, y_pt, z_pt)
platform_position = FK(theta1, theta2, theta3)

print("Desired Trajectory Point: ", (x_pt, y_pt, z_pt))
print("Computed Platform Position: ", platform_position)
plot_robot(theta1, theta2, theta3)

animate_robot((x_circle, y_circle, z_circle))